In [1]:
## Import the required libraries 

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from keras.utils.data_utils import GeneratorEnqueuer
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import math, os

%matplotlib inline

Using TensorFlow backend.
/anaconda3/envs/p3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/p3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
image_path = "./images/"

batch_size = 100
img_generator = ImageDataGenerator().flow_from_directory(image_path, shuffle=False, batch_size = batch_size)
n_rounds = math.ceil(img_generator.samples / img_generator.batch_size)
filenames = img_generator.filenames

img_generator = GeneratorEnqueuer(img_generator)
img_generator.start()
img_generator = img_generator.get()

Found 5 images belonging to 1 classes.


In [3]:
from imageai.Detection import ObjectDetection
model_weight_path = "resnet50_coco_best_v2.0.1.h5"

execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(model_weight_path)
detector.loadModel()

In [ ]:
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "./images/test/0a0a615629231821.jpg"), output_image_path=os.path.join(execution_path , "0a0a615629231821new.jpg"), minimum_percentage_probability=50)

In [16]:
for eachObject in detections:
    print(str(eachObject["name"]) + " " + str(eachObject["percentage_probability"]) + " " + str(eachObject["box_points"][0]) + " " + str(eachObject["box_points"][1]) + " " + str(eachObject["box_points"][2]) + " " + str(eachObject["box_points"][3]))

car 83.84596705436707 420 546 460 562
car 81.7827045917511 505 543 546 561
car 81.04318976402283 551 546 586 562
car 79.88312244415283 772 545 815 563
car 88.7209951877594 56 605 99 631
car 91.74057841300964 210 651 258 679
